Las Vegas (LV) Algorithms- Are randomized algorithms which always give the correct answer. The running time however is not fixed (not deterministic), that is it can vary for the same input. For eg. Randomized Quick Sort always gives a correctly sorted array as its output. However it takes $O(n\,logn)$ time on average but can be as bad as $O(n^2)$ in the worst case.

Monte Carlo (MC)  Algorithms - Are randomized algorithms which may not always give the right answer. The running time for these algorithms is fixed however. There is also generally a probability guarantee that the answer is right. For eg. if you used a non perfect hash to assign hash values to two different strings and then try to see if the strings are the same or not by comparing the hash values, then this is like a MC algorithm. You will mostly get the right answer but sometimes two different strings can end up having the same hash value.

Идея алгоритма Лас-Вегаса состоит в следующем. Если у нас есть некий вероятностный алгоритм $A$, 
который с определенной вероятностью дает верный результат, и существует возможность алгоритмически проверить результат алгоритма  $A$ на корректность (скажем, с помощью алгоритма $K$), 
то можно выполнять алгоритм A до тех пор, пока проверка не установит, что результат верен.

$$
Importance \ sampling
$$

$$
\int{f\mathrm{dV}} = \int{\left( \frac{f}{g} \right) g \mathrm{dV}} = 
\int{h g \mathrm{dV}}
$$

In [1]:
import numpy as np

In [2]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

# Examples

In [3]:
## exact ans is =560

def func(x):
    return x[:, 0]**2 + x[:, 1]**2 - 2 * x[:, 2]**2
fbound = np.array([-1, 3, -1, 6, -1, 4])

In [24]:
## exact ans is 39007.8

def func(x):
    return 2 * np.exp(x[:, 0]) * np.exp(x[:, 1]) +3 * np.exp(-2*x[:, 2])
fbound = np.array([-1, 3, -5, 4, -3, 5])

In [50]:
## exact ans is 18862.0

def func(x):
    return np.exp(x[:, 0]) * np.exp(x[:, 1]) - 30
fbound = np.array([-2, 4, -2, 4, 0, 10])

In [ ]:
## exact ans is 2415.6

def func(x):
    return 2 * np.exp(x[:, 0]) * np.exp(x[:, 1]) + 3 * np.exp(-2*x[:, 2]) - 30
fbound = np.array([-2, 3, -2, 3, -2, 6])

In [57]:
"""
    Находит интеграл функции по алгоритму Вегас.

    Parameters
    ---------------
    func : function
        - исходная функция
    ndim : int number
        - количество измерений
    fbound : numpy.ndarray
        - массив границ интегрирования. Размер 2*ndim 
    ndot, ntimes : int number
        - Каждый интеграл вычисляется в ndot точках ntimes раз. Необходимо для рассчета
        интеграла методом Монте-Карло и составления статистической выборки из результатов интегрирования.
    
    Returns
    -------
    intBest : double number
        - наилучшее исчисление интеграла.
        
    Examples
    --------
    >> las_vegas_baby(lambda x: x[:, 0]**2 + x[:, 1]**2 - 1, 3, np.array([-1, 1, -1, 1, -1, 1]), 20000,200)
    
    out: (-2.6744828047074023, 0.0045140421646385452)

    Inner variables
    -------
    g : fucntion
        - PDF
    integrals : numpy.ndarray
        - массив значений интеграла на каждом шаге вычислений
    standarddivs : numpy.ndarray
        - массив значений отклонений от среднего на каждом шаге вычислений
    grig : numpy.ndarray
        - массив начальных точек разбиения всей области интегрирования на ndot объемов
    gridlen : numpy.ndarray
        - длина одного интервала в каждом из измерений        
    weights : numpy.ndarray
        - массив весовых коэффициентов каждого из объемов

"""
##initializating probability density function g, grid for integration volume and weights for grid elements
def init(ndim, fbound, ndot, nperdim):
    vol = fbound[1::2] - fbound[0::2]
    
    size = np.prod(vol)
    if size <= 0:
            raise ValueError("size <= 0. Something went wrong!")

    def g(x):
        return 1.0/size
    
    grid = np.empty((ndim, nperdim))
    gridlen = np.empty(ndim)
    for i in range(ndim):
        wigth = vol[i]/(nperdim)
        grid[i] = np.linspace(fbound[2*i], fbound[2*i+1] - wigth, nperdim)
        gridlen[i] = wigth

    grid = cartesian(grid)       
    weights = np.ones(ndot)/ndot  
    
    return g, grid, gridlen, weights

## testing probability distribution function
def mc_dim_g_test(g, ndim, fbound, ndot):
    sum = 0
    vol = fbound[1::2] - fbound[0::2]    
    for i in range(ndot):
        x = np.empty(ndim)
        
        for j in range(ndim):
            x[j] = np.random.random() * vol[j] + fbound[2*j]
        sum += g(x)

    return np.prod(vol)*sum/ndot

## calculating integral of abs func
def mc_dim_abs(func, g, grid, gridlen, weights, ndim, fbound, ndot):
    sum = 0
    vol = fbound[1::2] - fbound[0::2]
    ind = []
    for i in range(ndot): ind.append(i)
        
    elem = np.random.choice(ind, ndot, p=weights)
    x = grid[elem] + np.random.random((ndot, ndim)) * gridlen   
    
    sum = np.sum(abs(func(x))/g(x))
    return sum / ndot

## calculating integral of func
def mc_dim(func, g, grid, gridlen, weights, ndim, fbound, ndot):
    sum = 0
    vol = fbound[1::2] - fbound[0::2]
    ind = []
    for i in range(ndot): ind.append(i)

    elem = np.random.choice(ind, ndot, p=weights)    
    x = grid[elem] + np.random.random((ndot, ndim)) * gridlen  

    sum = np.sum(func(x)/g(x))    
    return sum / ndot


## evolving probability distribution function g
def mutate_g(func, int_abs):
    def g(x):        
        return abs(func(x)) / int_abs
    return g


## evolving weights of our grid
def mutate_weights(g, grid, gridlen, weights, ndot):
    f_avr = np.empty(ndot)
    newgrid = grid + gridlen/2    
    f_avr = g(newgrid)
    #f_avr /= sum(f_avr)
    #f_avr += abs(np.random.normal(0, 1/ndot, ndot))    
    return f_avr/sum(f_avr)
            
    
    
 


    
    
## __main__ function   
def las_vegas_baby(func, ndim, fbound, ndot, ntimes):

    if not(fbound.size == (ndim * 2)):
        raise ValueError("Error in ndim-fbound size!")    
    
    # корень берется с ошибкой. Пришлось резать число ndot
    nperdim = int(ndot**(1/ndim))
    ndot = nperdim**ndim
    print("Actual number of dots is ", ndot)   
    if nperdim != int(nperdim):
        raise ValueError("Please let be ndot be ndim-root")        
    
    integrals = np.empty(0)
    standarddivs = np.empty(0)
    
    # at frst g is distpibuted uniformaly over volume
    g, grid, gridlen, weights = init(ndim, fbound, ndot, nperdim)    
    
    PDF_abs = np.empty(0)
    for _ in range(ntimes):
        PDF_abs = np.append(PDF_abs, mc_dim_abs(func, g, grid, gridlen, weights, ndim, fbound, ndot))  
   
    itmax = 4    
    for i in range(itmax):        
        PDF = np.empty(0)
        #PDF_abs = np.empty(0)

        for _ in range(ntimes):
            PDF = np.append(PDF, mc_dim(func, g, grid, gridlen, weights, ndim, fbound, ndot)) 
            #PDF_abs = np.append(PDF_abs, mc_dim_abs(func, g, grid, gridlen, weights, ndim, fbound, ndot))
            
            # make sure that \int{g\mathrm{dV}} = 1
            #PDF = np.append(PDF, mc_dim_tast(g, ndim, fbound, ndot))       
        integrals = np.append(integrals, np.mean(PDF))
        standarddivs = np.append(standarddivs, np.std(PDF))
       
        # mutate once
        if i<1:
            g = mutate_g(func, np.mean(PDF_abs))
            weights = mutate_weights(g, grid, gridlen, weights, ndot)   
        
        intBest = np.sum(integrals/standarddivs**2)/np.sum(1/standarddivs**2)
        stdBest = np.sum(1/standarddivs**2)**(-1/2)        
        
        ## print section
        print("--------------------", i+1, "--------------------")
        #rint("Value of integral = ", integrals[-1])
        #rint("Value of std = ", standarddivs[-1])
        #rint("|||||")
        print("Ans so far is ", intBest, "+-", stdBest)
        #rint("Best std is  ", stdBest)

        # Stop module
        
        #m = integrals.size
        #if m > 1:
        #    Xi = 1/(m-1) * np.sum(((integrals - intBest)/standarddivs)**2)
        #    print("Xi = ", Xi)
        #    if Xi != Xi:
        #        raise ValueError("Xi = NaN. Something went wrong!")
        #    #if Xi < 1:
        #    #    return intBest, stdBest
                   
    print("--------------------------------------------")    
    print("Answer = ", intBest, " +- ", stdBest)

    return intBest, stdBest

In [58]:
#%%timeit
ing, std = las_vegas_baby(func, 3, fbound, 30000, 300)

Actual number of dots is  29791


d:\Anaconda3\lib\site-packages\ipykernel\__main__.py:42: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
d:\Anaconda3\lib\site-packages\ipykernel\__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-------------------- 1 --------------------
Ans so far is  18855.0115041 +- 518.152534064
-------------------- 2 --------------------
Ans so far is  18862.3595287 +- 134.579359917
-------------------- 3 --------------------
Ans so far is  18866.9722071 +- 98.354908665
-------------------- 4 --------------------
Ans so far is  18863.624847 +- 82.5424837166
--------------------------------------------
Answer =  18863.624847  +-  82.5424837166
